<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 01: Feature Backfill for Air Quality Data</span>


## 🗒️ You have the following tasks
1. Choose an Air Quality Sensor
2. Update the country, city, and street information to point to YOUR chosen Air Quality Sensor
3. Download historical measures for your Air Quality Sensor as a CSV file
4. Update the path of the CSV file in this notebook to point to the one that you downloaded
5. Create an account on www.hopsworks.ai and get your HOPSWORKS_API_KEY
6. Run this notebook

### <span style='color:#ff5f27'> 📝 Imports

In [1]:
import datetime
import requests
import pandas as pd
import hopsworks
import datetime
from pathlib import Path
from functions import util
import json
import re
import os
import warnings
warnings.filterwarnings("ignore")

## Hopsworks API Key & AQI API Key
You need to have registered an account on app.hopsworks.ai to get Hopsworks API Key.

You can get your AQI API key [from here](https://aqicn.org/data-platform/token/)

Save AQI_API_KEY and HOPSWORKS_API_KEY to a `.env` file and add `.env` file to `.gitignore`:

 * <project root>/data/.env

In [2]:
from dotenv import load_dotenv
import os

env_path = '../../data/.env'
load_dotenv(dotenv_path=env_path)

HOPSWORKS_API_KEY = os.getenv("HOPSWORKS_API_KEY")
AQI_API_KEY = os.getenv("AQI_API_KEY")

## <span style='color:#ff5f27'> 🌍 STEP 1: Pick your Air Quality Sensor</span>

![alt text](./image/download.png)

  * Find your favorite sensor on https://waqi.info/ 
  * The sensor should have a URL in one of the two following forms: 

  `https://aqicn.org/station/<CITY OR COUNTRY NAME>/<STREET>`
  or

  `https://aqicn.org/station/@36655//`
  or 
  
  `https://aqicn.org/city/<CITY OR COUNTRY NAME>/<STREET>`

With your URL, we will need to do two things:

 * download the historical air quality readings as a CSV file
 * note down the URL for the real-time API (you will need to create an API key for accessing this).

If your sensor's URL has one of the first two formats (the first URL path component is `station`), you will find the links to both historical CSV data and the real-time API on the same web page.

However, if your sensor's URL has the last format above (the first URL path component is `city` instead of `station`), then you will need to use 2 different URLs - one to download the historical CSV data and one for the real-time air quality measurements. You will find both of those links in the "Air quality historical data" section. Click on the "Historical air quality data" when you need to download the CSV file, and when you need the real-time API, click on the "Real-time air quality data".

Some examples of URLs for stations:

 * https://aqicn.org/station/sweden/stockholm-hornsgatan-108-gata/ - in Stockholm, Sweden
 * https://aqicn.org/station/@36655// - in Hell's Kitchen, Manhatten, New York, USA
 * https://aqicn.org/station/nigeria-benin-city-edo-state-secretariat-palm-house/ - in Benin City, Nigeria
 * https://aqicn.org/station/india/mumbai/sion/ - Sion in Mumbai, India

Here is what the webpage at URL for the Stockholm sensor looks like:

![alt text](./image/download1.png)

__When you pick a sensor for your project, there are 2 things the sensor MUST have__:
  1. __PM 2.5__ measurements
  2. __Good Historical Values__ for download as a CSV file

__Write down the country, city, and the street for your sensor.__

We will use the city name to download weather data for your sensor, and we will store the country and street values in the sensor's feature group.

## What makes a good quality Air Quality Sensor?

In the image below, we can see below that our sensor in Stockholm fulfills our 2 requirements. It has:
  1. __PM 2.5__ measurements (see upper red-ringed value in image below)
  2. __Good Historical Measurements__ with few missing values (see lower red-ringed values in image below) 

![alt text](./image/download2.png)

---

## <span style='color:#ff5f27'> 🌍 STEP 2: Download the Historical Air Quality </span>

You can download a CSV file containing the historical air quality data from your your sensor's URL.
Scroll down to the section "Air Quality Historical Data". Click on the PM2.5 option and save the file to the `data` directory in your forked version of this Github repository. Note the name of your CSV file, you will need 

![alt text](./image/download3.png)

## <span style='color:#ff5f27'> 🌍 STEP 3: Get the AQICN_URL and API key. Enter country, city, street names for your Sensor.</span>

You can find your __AQICN_URL__ if you scroll down the webpage for your sensor - it is the URL inside the redbox here.
You shouldn't include the last part of the url - "/?token=\__YOUR_TOKEN\__". 
It is bad practice to save TOKENs (aka API KEYs) in your source code - you might make it public if you check that code into Github!
We will fill in the token later by saving the AQI_API_KEY as a secret in Hopsworks.

![alt text](./image/download4.png)

__Update the values in the cell below.__

In [3]:
csv_file="../../data/jing'an-monitoring station-air-quality.csv"
util.check_file_path(csv_file)

File successfully found at the path: ../../data/jing'an-monitoring station-air-quality.csv


In [4]:
country="china"
city = "shanghai"
street = "jinganjiancezhan"
aqicn_url="https://api.waqi.info/feed/@486"

latitude, longitude = util.get_city_coordinates(city)
today = datetime.date.today()
latitude, longitude, today

(31.23, 121.47, datetime.date(2024, 11, 13))

## <span style='color:#ff5f27'> 🌍 STEP 4: Store AQI_API_KEY as a Secret in Hopsworks </span>

In [5]:
project = hopsworks.login(project='mlfs_lab1')

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1163420


In [6]:
secrets = util.secrets_api(project.name)
try:
    secrets.create_secret("AQI_API_KEY", AQI_API_KEY)
except hopsworks.RestAPIError:
    AQI_API_KEY = secrets.get_secret("AQI_API_KEY").value

Connected. Call `.close()` to terminate connection gracefully.


### Validate that the AQI_API_KEY you added earlier works

The cell below should print out something like:

![alt text](./image/download5.png)

In [7]:
try:
    aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQI_API_KEY)
except hopsworks.RestAPIError:
    print("It looks like the AQI_API_KEY doesn't work for your sensor. Is the API key correct? Is the sensor URL correct?")

aq_today_df.head()

,pm25,country,city,street,date,url
0,72.0,china,shanghai,jinganjiancezhan,2024-11-13,https://api.waqi.info/feed/@486


## <span style='color:#ff5f27'> 🌍 STEP 5: Read your CSV file into a DataFrame </span>

The cell below will read up historical air quality data as a CSV file into a Pandas DataFrame

## Check the data types for the columns in your DataFrame

 * `date` should be of type   datetime64[ns] 
 * `pm25` should be of type float64

In [8]:
df = pd.read_csv(csv_file,  parse_dates=['date'], skipinitialspace=True)
df

,date,pm25,pm10,o3,no2,so2,co
0,2024-11-01,20.0,45.0,44.0,13.0,3.0,5.0
1,2024-11-02,103.0,58.0,47.0,19.0,3.0,6.0
2,2024-11-03,140.0,48.0,58.0,18.0,4.0,6.0
3,2024-11-04,124.0,21.0,31.0,10.0,3.0,3.0
4,2024-11-05,45.0,20.0,35.0,12.0,3.0,3.0
...,...,...,...,...,...,...,...
3957,2018-12-31,NaN,33.0,27.0,10.0,4.0,5.0
3958,2017-09-10,NaN,31.0,28.0,18.0,4.0,11.0
3959,2016-01-03,NaN,95.0,30.0,35.0,11.0,10.0
3960,2014-12-31,NaN,61.0,21.0,21.0,16.0,6.0


## <span style='color:#ff5f27'> 🌍 STEP 6: Data cleaning</span>


### Rename columns if needed and drop unneccessary columns

We want to have a DataFrame with 2 columns - `date` and `pm25` after this cell below:

## Check the data types for the columns in your DataFrame

 * `date` should be of type   datetime64[ns] 
 * `pm25` should be of type float64

In [9]:
# These commands will succeed if your CSV file didn't have a `median` or `timestamp` column
df = df.rename(columns={"median": "pm25"})
df = df.rename(columns={"timestamp": "date"})

df_aq = df[['date', 'pm25']]
df_aq['pm25'] = df_aq['pm25'].astype('float32')
df_aq

,date,pm25
0,2024-11-01,20.0
1,2024-11-02,103.0
2,2024-11-03,140.0
3,2024-11-04,124.0
4,2024-11-05,45.0
...,...,...
3957,2018-12-31,NaN
3958,2017-09-10,NaN
3959,2016-01-03,NaN
3960,2014-12-31,NaN


In [10]:
# Cast the pm25 column to be a float32 data type
df_aq.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3962 entries, 0 to 3961
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    3962 non-null   datetime64[ns]
 1   pm25    3955 non-null   float32       
dtypes: datetime64[ns](1), float32(1)
memory usage: 46.6 KB


## <span style='color:#ff5f27'> 🌍 STEP 7: Drop any rows with missing data </span>
It will make the model training easier if there is no missing data in the rows, so we drop any rows with missing data.

In [11]:
df_aq.dropna(inplace=True)
df_aq

,date,pm25
0,2024-11-01,20.0
1,2024-11-02,103.0
2,2024-11-03,140.0
3,2024-11-04,124.0
4,2024-11-05,45.0
...,...,...
3950,2014-03-27,108.0
3951,2014-03-28,103.0
3952,2014-03-29,153.0
3953,2014-03-30,173.0


## <span style='color:#ff5f27'> 🌍 STEP 8: Add country, city, street, url to the DataFrame </span>

Your CSV file may have many other air quality measurement columns. We will only work with the `pm25` column.

We add the columns for the country, city, and street names that you changed for your Air Quality sensor.

We also want to make sure the `pm25` column is a float32 data type.

In [12]:
# Your sensor may have columns we won't use, so only keep the date and pm25 columns
# If the column names in your DataFrame are different, rename your columns to `date` and `pm25`
df_aq['country']=country
df_aq['city']=city
df_aq['street']=street
df_aq['url']=aqicn_url

df_aq.sort_values(by='date', ascending=True).tail(20)

,date,pm25,country,city,street,url
37,2024-10-25,47.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
38,2024-10-26,23.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
39,2024-10-27,34.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
40,2024-10-28,115.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
41,2024-10-29,49.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
42,2024-10-30,54.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
43,2024-10-31,49.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
0,2024-11-01,20.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
1,2024-11-02,103.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
2,2024-11-03,140.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486


## <span style='color:#ff5f27'> 🌦 Loading Weather Data from [Open Meteo](https://open-meteo.com/en/docs)

## <span style='color:#ff5f27'> 🌍 STEP 9: Download the Historical Weather Data </span>

https://open-meteo.com/en/docs/historical-weather-api#hourly=&daily=temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant

We will download the historical weather data for your `city` by first extracting the earliest date from your DataFrame containing the historical air quality measurements.

We will download all daily historical weather data measurements for your `city` from the earliest date in your air quality measurement DataFrame. It doesn't matter if there are missing days of air quality measurements. We can store all of the daily weather measurements, and when we build our training dataset, we will join up the air quality measurements for a given day to its weather features for that day. 

The weather features we will download are:

 * `temperature (average over the day)`
 * `precipitation (the total over the day)`
 * `wind speed (average over the day)`
 * `wind direction (the most dominant direction over the day)`


In [23]:
earliest_aq_date = pd.Series.min(df_aq['date'])
earliest_aq_date = earliest_aq_date.strftime('%Y-%m-%d')
earliest_aq_date

weather_df = util.get_historical_weather(city, earliest_aq_date, str(today), latitude, longitude)

hourly_df = util.get_yesterday_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city

weather_df = pd.concat([weather_df, daily_df.head(1)])

for i in range(1, 22):
    df_temp = df_aq.copy()
    df_temp['date'] = pd.to_datetime(df_temp['date']) + pd.Timedelta(days=i)
    weather_df = pd.merge(weather_df, df_temp[['date','pm25']], how='left', on="date")
    weather_df = weather_df.rename(columns={"pm25": f"pm25_{i}"})

weather_df.dropna(inplace=True)

weather_df.sort_values(by='date', ascending=True).tail(10)

Coordinates 31.247802734375°N 121.5°E
Elevation 3.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city,pm25_1,pm25_2,pm25_3,pm25_4,...,pm25_12,pm25_13,pm25_14,pm25_15,pm25_16,pm25_17,pm25_18,pm25_19,pm25_20,pm25_21
3959,2024-11-03,17.740332,0.0,8.149847,16.130766,shanghai,103.0,20.0,49.0,54.0,...,104.0,82.0,48.0,52.0,73.0,68.0,71.0,108.0,60.0,43.0
3960,2024-11-04,17.409084,0.0,19.645580,10.283305,shanghai,140.0,103.0,20.0,49.0,...,95.0,104.0,82.0,48.0,52.0,73.0,68.0,71.0,108.0,60.0
3961,2024-11-05,13.817414,0.0,19.852295,358.205353,shanghai,124.0,140.0,103.0,20.0,...,46.0,95.0,104.0,82.0,48.0,52.0,73.0,68.0,71.0,108.0
3962,2024-11-06,13.725750,0.0,17.589418,15.969136,shanghai,45.0,124.0,140.0,103.0,...,47.0,46.0,95.0,104.0,82.0,48.0,52.0,73.0,68.0,71.0
3963,2024-11-07,14.234082,0.0,14.450535,34.656887,shanghai,35.0,45.0,124.0,140.0,...,23.0,47.0,46.0,95.0,104.0,82.0,48.0,52.0,73.0,68.0
3964,2024-11-08,17.711164,0.0,18.275984,44.213470,shanghai,35.0,35.0,45.0,124.0,...,34.0,23.0,47.0,46.0,95.0,104.0,82.0,48.0,52.0,73.0
3965,2024-11-09,18.850750,0.5,21.862406,43.717545,shanghai,55.0,35.0,35.0,45.0,...,115.0,34.0,23.0,47.0,46.0,95.0,104.0,82.0,48.0,52.0
3966,2024-11-10,16.086168,1.2,18.553974,0.802675,shanghai,46.0,55.0,35.0,35.0,...,49.0,115.0,34.0,23.0,47.0,46.0,95.0,104.0,82.0,48.0
3967,2024-11-11,16.271584,0.0,14.092977,25.547438,shanghai,61.0,46.0,55.0,35.0,...,54.0,49.0,115.0,34.0,23.0,47.0,46.0,95.0,104.0,82.0
3968,2024-11-12,17.000000,0.0,5.091168,44.999897,shanghai,87.0,61.0,46.0,55.0,...,49.0,54.0,49.0,115.0,34.0,23.0,47.0,46.0,95.0,104.0


In [14]:
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3820 entries, 20 to 3968
Data columns (total 26 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         3820 non-null   datetime64[ns]
 1   temperature_2m_mean          3820 non-null   float32       
 2   precipitation_sum            3820 non-null   float32       
 3   wind_speed_10m_max           3820 non-null   float32       
 4   wind_direction_10m_dominant  3820 non-null   float32       
 5   city                         3820 non-null   object        
 6   pm25_1                       3820 non-null   float32       
 7   pm25_2                       3820 non-null   float32       
 8   pm25_3                       3820 non-null   float32       
 9   pm25_4                       3820 non-null   float32       
 10  pm25_5                       3820 non-null   float32       
 11  pm25_6                       3820 non-null   fl

## <span style='color:#ff5f27'> 🌍 STEP 10: Define Data Validation Rules </span>

We will validate the air quality measurements (`pm25` values) before we write them to Hopsworks.

We define a data validation rule (an expectation in Great Expectations) that ensures that `pm25` values are not negative or above the max value available by the sensor.

We will attach this expectation to the air quality feature group, so that we validate the `pm25` data every time we write a DataFrame to the feature group. We want to prevent garbage-in, garbage-out.

In [15]:
import great_expectations as ge
aq_expectation_suite = ge.core.ExpectationSuite(
    expectation_suite_name="aq_expectation_suite"
)

aq_expectation_suite.add_expectation(
    ge.core.ExpectationConfiguration(
        expectation_type="expect_column_min_to_be_between",
        kwargs={
            "column":"pm25",
            "min_value":-0.1,
            "max_value":500.0,
            "strict_min":True
        }
    )
)

{"expectation_type": "expect_column_min_to_be_between", "kwargs": {"column": "pm25", "min_value": -0.1, "max_value": 500.0, "strict_min": true}, "meta": {}}

## Expectations for Weather Data
Here, we define an expectation for 2 columns in our weather DataFrame - `precipitation_sum` and `wind_speed_10m_max`, where we expect both values to be greater than zero, but less than 1000.

In [28]:
import great_expectations as ge
weather_expectation_suite = ge.core.ExpectationSuite(
    expectation_suite_name="weather_expectation_suite"
)

def expect_greater_than_zero(col):
    weather_expectation_suite.add_expectation(
        ge.core.ExpectationConfiguration(
            expectation_type="expect_column_min_to_be_between",
            kwargs={
                "column":col,
                "min_value":-0.1,
                "max_value":1000.0,
                "strict_min":True
            }
        )
    )
expect_greater_than_zero("precipitation_sum")
expect_greater_than_zero("wind_speed_10m_max")
for i in range(1, 22):
    expect_greater_than_zero(f"pm25_{i}")

---

### <span style="color:#ff5f27;"> 🔮 STEP 11: Connect to Hopsworks and save the sensor country, city, street names as a secret</span>

In [17]:
fs = project.get_feature_store() 

2024-11-13 21:25:08,559 WARNING: using legacy validation callback
Connected. Call `.close()` to terminate connection gracefully.


#### Save country, city, street names as a secret

These will be downloaded from Hopsworks later in the (1) daily feature pipeline and (2) the daily batch inference pipeline

In [18]:
dict_obj = {
    "country": country,
    "city": city,
    "street": street,
    "aqicn_url": aqicn_url,
    "latitude": latitude,
    "longitude": longitude
}

# Convert the dictionary to a JSON string
str_dict = json.dumps(dict_obj)

try:
    secrets.create_secret("SENSOR_LOCATION_JSON", str_dict)
except hopsworks.RestAPIError:
    print("SENSOR_LOCATION_JSON already exists. To update, delete the secret in the UI (https://c.app.hopsworks.ai/account/secrets) and re-run this cell.")
    existing_key = secrets.get_secret("SENSOR_LOCATION_JSON").value
    print(f"{existing_key}")

SENSOR_LOCATION_JSON already exists. To update, delete the secret in the UI (https://c.app.hopsworks.ai/account/secrets) and re-run this cell.
{"country": "china", "city": "shanghai", "street": "jinganjiancezhan", "aqicn_url": "https://api.waqi.info/feed/@486", "latitude": 31.23, "longitude": 121.47}


### <span style="color:#ff5f27;"> 🔮 STEP 12: Create the Feature Groups and insert the DataFrames in them </span>

### <span style='color:#ff5f27'> 🌫 Air Quality Data
    
 1. Provide a name, description, and version for the feature group.
 2. Define the `primary_key`: we have to select which columns uniquely identify each row in the DataFrame - by providing them as the `primary_key`. Here, each air quality sensor measurement is uniquely identified by `country`, `street`, and  `date`.
 3. Define the `event_time`: We also define which column stores the timestamp or date for the row - `date`.
 4. Attach any `expectation_suite` containing data validation rules

In [21]:
air_quality_fg = fs.get_or_create_feature_group(
    name='air_quality',
    description='Air Quality characteristics of each day',
    version=3,
    primary_key=['city', 'street', 'date'],
    event_time="date",
    expectation_suite=aq_expectation_suite
)

#### Insert the DataFrame into the Feature Group

In [22]:
air_quality_fg.insert(df_aq, write_options={"wait_for_job": True})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1163420/fs/1154123/fg/1347016
2024-11-13 21:31:26,935 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1163420/fs/1154123/fg/1347016


Uploading Dataframe: 0.00% |          | Rows 0/3955 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: air_quality_3_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1163420/jobs/named/air_quality_3_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x311c60830>,
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 670848
         }
       },
       "result": {
         "observed_value": 11.0,
         "element_count": 3955,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-11-13T08:31:26.000934Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "successful_expectations": 1,
     "uns

#### Enter a description for each feature in the Feature Group

In [25]:
air_quality_fg.update_feature_description("date", "Date of measurement of air quality")
air_quality_fg.update_feature_description("country", "Country where the air quality was measured (sometimes a city in acqcn.org)")
air_quality_fg.update_feature_description("city", "City where the air quality was measured")
air_quality_fg.update_feature_description("street", "Street in the city where the air quality was measured")
air_quality_fg.update_feature_description("pm25", "Particles less than 2.5 micrometers in diameter (fine particles) pose health risk")

In [27]:
display(air_quality_fg.read().sort_values(by=['date'], ascending=True).tail(10))

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.70s) 


,date,pm25,country,city,street,url
540,2024-11-04 00:00:00+00:00,124.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
348,2024-11-05 00:00:00+00:00,45.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
1287,2024-11-06 00:00:00+00:00,35.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
1371,2024-11-07 00:00:00+00:00,35.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
3122,2024-11-08 00:00:00+00:00,55.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
2556,2024-11-09 00:00:00+00:00,46.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
3040,2024-11-10 00:00:00+00:00,61.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
3217,2024-11-11 00:00:00+00:00,87.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
3785,2024-11-12 00:00:00+00:00,93.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
779,2024-11-13 00:00:00+00:00,115.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486


### <span style='color:#ff5f27'> 🌦 Weather Data
    
 1. Provide a name, description, and version for the feature group.
 2. Define the `primary_key`: we have to select which columns uniquely identify each row in the DataFrame - by providing them as the `primary_key`. Here, each weather measurement is uniquely identified by `city` and  `date`.
 3. Define the `event_time`: We also define which column stores the timestamp or date for the row - `date`.
 4. Attach any `expectation_suite` containing data validation rules

In [26]:
# Get or create feature group 
weather_fg = fs.get_or_create_feature_group(
    name='weather',
    description='Weather characteristics of each day',
    version=5,
    primary_key=['city', 'date'],
    event_time="date",
    expectation_suite=weather_expectation_suite
) 

#### Insert the DataFrame into the Feature Group

In [29]:
# Insert data
weather_fg.insert(weather_df, write_options={"wait_for_job": True})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1163420/fs/1154123/fg/1347018
2024-11-13 22:09:38,686 INFO: 	22 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1163420/fs/1154123/fg/1347018


Uploading Dataframe: 0.00% |          | Rows 0/3813 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: weather_5_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1163420/jobs/named/weather_5_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x3121db8c0>,
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25_11",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 670851
         }
       },
       "result": {
         "observed_value": 11.0,
         "element_count": 3813,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-11-13T09:09:38.000685Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
    

#### Enter a description for each feature in the Feature Group

In [30]:
weather_fg.update_feature_description("date", "Date of measurement of weather")
weather_fg.update_feature_description("city", "City where weather is measured/forecast for")
weather_fg.update_feature_description("temperature_2m_mean", "Temperature in Celsius")
weather_fg.update_feature_description("precipitation_sum", "Precipitation (rain/snow) in mm")
weather_fg.update_feature_description("wind_speed_10m_max", "Wind speed at 10m abouve ground")
weather_fg.update_feature_description("wind_direction_10m_dominant", "Dominant Wind direction over the dayd")
for i in range(1, 22):
    weather_fg.update_feature_description(f"pm25_{i}", "Particles less than 2.5 micrometers in diameter (fine particles) pose health risk")

In [31]:
display(weather_fg.read().sort_values(by=['date'], ascending=True).tail(10))

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.40s) 


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city,pm25_1,pm25_2,pm25_3,pm25_4,...,pm25_12,pm25_13,pm25_14,pm25_15,pm25_16,pm25_17,pm25_18,pm25_19,pm25_20,pm25_21
2247,2024-11-03 00:00:00+00:00,17.740332,0.0,8.149847,16.130766,shanghai,103.0,20.0,49.0,54.0,...,104.0,82.0,48.0,52.0,73.0,68.0,71.0,108.0,60.0,43.0
3670,2024-11-04 00:00:00+00:00,17.409084,0.0,19.645580,10.283305,shanghai,140.0,103.0,20.0,49.0,...,95.0,104.0,82.0,48.0,52.0,73.0,68.0,71.0,108.0,60.0
2822,2024-11-05 00:00:00+00:00,13.817414,0.0,19.852295,358.205353,shanghai,124.0,140.0,103.0,20.0,...,46.0,95.0,104.0,82.0,48.0,52.0,73.0,68.0,71.0,108.0
1212,2024-11-06 00:00:00+00:00,13.725750,0.0,17.589418,15.969136,shanghai,45.0,124.0,140.0,103.0,...,47.0,46.0,95.0,104.0,82.0,48.0,52.0,73.0,68.0,71.0
2441,2024-11-07 00:00:00+00:00,14.234082,0.0,14.450535,34.656887,shanghai,35.0,45.0,124.0,140.0,...,23.0,47.0,46.0,95.0,104.0,82.0,48.0,52.0,73.0,68.0
2235,2024-11-08 00:00:00+00:00,17.711164,0.0,18.275984,44.213470,shanghai,35.0,35.0,45.0,124.0,...,34.0,23.0,47.0,46.0,95.0,104.0,82.0,48.0,52.0,73.0
1078,2024-11-09 00:00:00+00:00,18.850750,0.5,21.862406,43.717545,shanghai,55.0,35.0,35.0,45.0,...,115.0,34.0,23.0,47.0,46.0,95.0,104.0,82.0,48.0,52.0
956,2024-11-10 00:00:00+00:00,16.086168,1.2,18.553974,0.802675,shanghai,46.0,55.0,35.0,35.0,...,49.0,115.0,34.0,23.0,47.0,46.0,95.0,104.0,82.0,48.0
3166,2024-11-11 00:00:00+00:00,16.271584,0.0,14.092977,25.547438,shanghai,61.0,46.0,55.0,35.0,...,54.0,49.0,115.0,34.0,23.0,47.0,46.0,95.0,104.0,82.0
838,2024-11-12 00:00:00+00:00,17.000000,0.0,5.091168,44.999897,shanghai,87.0,61.0,46.0,55.0,...,49.0,54.0,49.0,115.0,34.0,23.0,47.0,46.0,95.0,104.0


## <span style="color:#ff5f27;">⏭️ **Next:** Part 02: Daily Feature Pipeline 
 </span> 


## <span style="color:#ff5f27;">⏭️ **Exercises:** 
 </span> 

Extra Homework:

  * Try adding a new feature based on a rolling window of 3 days for 'pm25'
      * This is not easy, as forecasting more than 1 day in the future, you won't have the previous 3 days of pm25 measurements.
      * df.set_index("date").rolling(3).mean() is only the start....
  * Parameterize the notebook, so that you can provide the `country`/`street`/`city`/`url`/`csv_file` as parameters. 
      * Hint: this will also require making the secret name (`SENSOR_LOCATION_JSON`), e.g., add the street name as part of the secret name. Then you have to pass that secret name as a parameter when running the operational feature pipeline and batch inference pipelines.
      * After you have done this, collect the street/city/url/csv files for all the sensors in your city or region and you make dashboards for all of the air quality sensors in your city/region. You could even then add a dashboard for your city/region, as done [here for Poland](https://github.com/erno98/ID2223).

Improve this AI System
  * As of mid 2024, there is no API call available to download historical data from the AQIN website. You could improve this system by writing a PR to download the CSV file using Python Selenium and the URL for the sensor.
